## Ejemplo de las valoraciones de un restaurante


In [3]:
import pandas as pd
import numpy as np

# Create a small sample dataset
X_train = pd.DataFrame(data={'review': ['Good food good service.', 
                                        'Good food with friendly service.',
                                        'Average food and bad service'],
                             'star': [5,4,2],
                             'meal_time': ['dinner','lunch','breakfirst'],
                             'tip_%': [0.25, 0.18, np.nan]})
X_train.head()

,review,star,meal_time,tip_%
0,Good food good service.,5,dinner,0.25
1,Good food with friendly service.,4,lunch,0.18
2,Average food and bad service,2,breakfirst,NaN


### Construimos un ColumTransformer

1. Utiliza OneHotEncoder para codificar la columna meal_time
2. Crea una pipeline para imputar los valores nulos de las columnas numéricas star y tip_% y escalarlas linealmente con MinMaxScaler
3. El resto de columnas (review) déjalas pasar sin modificaciones
4. Combina ambas lineas con un ColumnTransformer

In [6]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

cat_col = ['meal_time']
num_col = ['star','tip_%']

# make a pipeline to do computing and scaling
num_pipe = Pipeline([
    ('computer',SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', MinMaxScaler())
])

# construct the ColumnTransformer
ColumnTransformation = ColumnTransformer(
    transformers=[
        # one-hot encode categorical cols 
        ('cat_ohe', OneHotEncoder(sparse_output = False, handle_unknown='ignore'), cat_col),
        # pipe transform numeric cols
        ('num_pipe', num_pipe, num_col)
    ]
        # passthrough the rest cols
    , remainder = 'passthrough'
)

# fit and transform the data
ColumnTransformation.fit_transform(X_train)

array([[0.0, 1.0, 0.0, 0.9999999999999999, 1.0,
        'Good food good service.'],
       [0.0, 0.0, 1.0, 0.6666666666666666, 0.72,
        'Good food with friendly service.'],
       [1.0, 0.0, 0.0, 0.0, 0.0, 'Average food and bad service']],
      dtype=object)

## Scikit-Learn ColumnTransformer
Extraído de la [web](https://gist.github.com/iamirmasoud/03b8788e87768691103784af9767cbd8)

In [7]:
from seaborn import load_dataset
#Set seed
seed = 123

#Loading data sets
df = load_dataset('tips').drop(columns=['tip', 'sex']).sample(n=5, random_state=seed)
 
#Add missing values
df.iloc[[1, 2, 4], [2, 4]] = np.nan
df


,total_bill,smoker,day,time,size
112,38.07,No,Sun,Dinner,3.0
19,20.65,No,NaN,Dinner,NaN
187,30.46,Yes,NaN,Dinner,NaN
169,10.63,Yes,Sat,Dinner,2.0
31,18.35,No,NaN,Dinner,NaN


In [8]:
from sklearn.model_selection import train_test_split
# Partition data
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['total_bill']),
                                                    df['total_bill'],
                                                    test_size=.2,
                                                    random_state=seed)

# Define classification columns
categorical = list(X_train.select_dtypes('category').columns)
print(f"Categorical columns are: {categorical}")

# Define numeric columns
numerical = list(X_train.select_dtypes('number').columns)
print(f"Numerical columns are: {numerical}")


Categorical columns are: ['smoker', 'day', 'time']
Numerical columns are: ['size']


<img src="data/columntransformer.png" width="400" />

In [11]:
from sklearn.linear_model import LinearRegression

# Define classification pipeline
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

# Define value pipeline
num_pipe = Pipeline([('imputer', SimpleImputer(strategy='median')),
                     ('scaler', MinMaxScaler())])

# Combined classification pipeline and numerical pipeline
preprocessor = ColumnTransformer(transformers=[('cat', cat_pipe, categorical),
                                               ('num', num_pipe, numerical)])

# Install transformer and training data estimator on the pipeline
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('model', LinearRegression())])
pipe.fit(X_train, y_train)

# Forecast training data
y_train_pred = pipe.predict(X_train)
print(f"Predictions on training data: {y_train_pred}")

# Forecast test data
y_test_pred = pipe.predict(X_test)
print(f"Predictions on test data: {y_test_pred}")


Predictions on training data: [10.63 18.35 38.07 30.46]
Predictions on test data: [18.35]


## Scikit-Learn FeatureUnion
Podemos pensar en FeatureUnion como si creara una copia de los datos, las transformara en paralelo y luego pegara los resultados. El término "copia" es más una analogía para facilitar la conceptualización que una referencia técnica.

Al inicio de cada pipeline, añadimos un paso adicional donde seleccionamos las columnas relevantes mediante un transformador personalizado Así es como podemos resumir visualmente el script anterior:

<img src="data/featureunion.png" width="400" />

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
# Custom pipe
class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""

    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.columns]


# Define classification pipeline
cat_pipe = Pipeline([('selector', ColumnSelector(categorical)),
                     ('imputer', SimpleImputer(
                         strategy='constant', fill_value='missing')),
                     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

# Define value pipeline
num_pipe = Pipeline([('selector', ColumnSelector(numerical)),
                     ('imputer', SimpleImputer(strategy='median')),
                     ('scaler', MinMaxScaler())])

# Combined classification pipeline and numerical pipeline
preprocessor = FeatureUnion(transformer_list=[('cat', cat_pipe),
                                              ('num', num_pipe)])

# Combined classification pipeline and numerical pipeline
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('model', LinearRegression())])
pipe.fit(X_train, y_train)

# Forecast training data
y_train_pred = pipe.predict(X_train)
print(f"Predictions on training data: {y_train_pred}")

# Forecast test data
y_test_pred = pipe.predict(X_test)
print(f"Predictions on test data: {y_test_pred}")


Predictions on training data: [10.63 18.35 38.07 30.46]
Predictions on test data: [18.35]


## Comparativa entre FeatureUnion y ColumnTransformer
Ambos métodos se utilizan para combinar transformaciones independientes (transformadores) en un único transformador. Por independientes me refiero a transformaciones que no necesitan ejecutarse secuencialmente, sino que se ejecutan en paralelo y la salida de cada transformación se fusiona al final.

La principal diferencia radica en que cada transformador en un objeto de unión de características recibe como entrada todo el conjunto de datos. Mientras que en el objeto de transformador de columnas, solo reciben como entrada una parte de los datos. FeatureUnion aplica diferentes transformadores a todos los datos de entrada y luego combina los resultados concatenándolos. ColumnTransformer, por otro lado, aplica diferentes transformadores a diferentes subconjuntos de todos los datos de entrada y, a su vez, concatena los resultados.

Como se vio en el ejemplo anterior, usar FeatureUnion es más complejo que usar ColumnTransformer. Por lo tanto, en mi opinión, es mejor usar ColumnTransformer en un caso similar. Sin embargo, FeatureUnion definitivamente tiene su lugar. Si alguna vez necesita transformar los mismos datos de entrada de diferentes maneras y usarlos como características, FeatureUnion es la herramienta ideal. Por ejemplo, si trabaja con datos de texto y desea vectorizarlos mediante tf-idf y extraer la longitud del texto, FeatureUnion es la herramienta perfecta.

# Ejercicios
1. Construir un FunctionalTransformer que elimine las filas duplicadas de una matriz
2. Construir un StandardScalerCustom que resista el test check_estimator(StandardScalerCustom())
3. Construir un Transformer que elimine las filas de un DataFrame que superen un porcentaje determinado de valores nulos o en las que la etiqueta tenga un valor nulo.
4. Crear un transformer que convierta las columnas que tu le dices a categóricas sino lo son ya creando imputando el valor más frecuente en los valores nulos.
5. Construir un transformer que:
   1. Actúa sólo en columnas numéricas. Si son categóricas se asegura de que tengan más de una categoría.
   2. Si solo hay un valor único **elimina** la columna
   3. Si el porcentaje de valores únicos es inferior al x% (será un parámetro) y hay menos de y (será otro parámetro) valores únicos **sustituye** la columna numérica por una categórica con tantas categorías como valores únicos
   4. Si no se cumple ninguna de las condiciones anteriores **añade** una columna categórica con una categoría por defecto y otra categoría para cada valor único cuyo cuyo porcentaje de recuentos está a más de n (será otro parámetro) desviaciones típicas de la media de procentajes de recuentos. Mantiene la columna numérica, por supuesto pero etiqueta la categórica según corresponda
   5. Después de llamar a fit devolverá una lista con las categorías que va a crear para cada columna
      - <-1: La columna numérica se **sustituirá** por una columna categórica e indica el número de categorías que tendrá
      - -1: La columna va a ser **eliminada** porque sólo tiene una categoría
      - 0: La columna es numérica y no se va a tocar.
      - 1: La columna es categórica y no se va a tocar
      - \>1: Se va a **añadir** una columna categórica e indica el número de categorías que tendrá